**AMP Incorporated Tree Packing** 

Quantum Computing is one of the most promising but clumsy technologies of today's world. Though critical applications are emerging in a wide variety of fields, the inefficiency of modern quantum processing units (QPUs) is holding them back from a more widespread utilization. One proposed solution for fighting this inefficiency is to run numerous instances of a provided quantum circuit at the same time by allocating space for multiple quantum circuits into a single QPU. For example, a circuit that requires 5 qubits running on a QPU with 12 available qubits leaves 7 unused, when instead, two of those circuits could run simultaneously and leave only 2 unused - not unlike parallelism in classical CPUs. 

**1. Problem Definition** 

Unfortunately, the problem is not as simple as figuring out how to fit as many circuits onto a single QPU as possible. 

Due to error prone qubits with poor connections that can appear after each recallibration of a QPU, it must be considered that some qubits will be better suited for a task than other qubits. Additionally, packing qubits too close to each other, especially those involved in CNOT operations, can cause unwanted crosstalk that results in significantly degraded performace. On the other hand, packing qubits too far away from each other runs the risk of wasting robust qubits and creating unreliably long qubit SWAP paths. Note that any greedy algorithm that simply selects the best qubits for each circuit at a time is subpar. 

So, to maximize efficiency, we should develop a packing algorithm that takes advantage of a QPU's toplogy to place as many circuits as possible, where each lacks a significant fidelity cost. 

**2. Solution: AMP Incorporated Tree Packing** 

Our proposed solution is to combine the strategies of creating a hierarchy tree for the QPU's available qubits, then searching bottom-up for an ideal cluster [1] and utilizing adaptive multi-programming (AMP) to decide on a case-by-case basis whether the available qubits are robust enough to run concurrent circuits with [2]. This way, if the available qubits and their connections are appropriate for the task at hand, the time it takes to reach the required shots could be reduced drastically by having multiple circuits contribute to the shot count at the same time. 


In [ ]:
#code

**3. Tradeoff Analysis** 

One crucial benefit of utilizing AMP is that we are never required to unwillingly sacrifice fidelity for shot savings. Should AMP be implemented into packing algorithms for general use, we reccomend that the cutoff for a multi-programming setup be adjustable by the user themselves. This way, the user can easily adjust the fidelity or shot savings they are aiming for if their program values speed or accuracy.

**References** (todo) 

[1]
[2] P. Das, S. S. Tannu, P. J. Nair, and M. Qureshi, “A Case for MultiProgramming Quantum Computers,” in Proceedings of the 52nd Annual IEEE/ACM 
    International Symposium on Microarchitecture (Micro), 2019.

 https://arxiv.org/pdf/2207.14483
 https://memlab.ece.gatech.edu/papers/MICRO_2019_4.pdf